[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/question-answering/table-qa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/question-answering/table-qa.ipynb)

# Table Question Answering with Pinecone

# Install Dependencies

# Load the Dataset

In [7]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/Final_Master_MAC_Centers_Data (1).csv')

# Convert all columns to string
df = df.astype(str)

# Store all tables in the tables list
tables = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Convert the row into a DataFrame with the header being the column names
    table = pd.DataFrame([row], columns=df.columns)
    tables.append(table)

In [16]:
tables[1741]

,Unnamed: 0,ID,Provider,AddressLine1,AddressLine2,City,State,PostalCode,Phone1_Type,Phone1_Number,...,Jefferson Davis,Lee,Covington,Harrison,Coahoma,Pearl River,Chickasaw,Monroe,Wilkinson,Pike
1741,1741,1742,Adoratiom Hospice,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Initialize Retriever

The retriever transforms natural language queries and tabular data into embeddings/vectors. It will generate embeddings in a way that the natural language questions and tables containing answers to our questions are nearby in the vector space.

We will use a SentenceTransformer model trained specifically for embedding tabular data for retrieval tasks. The model can be loaded from the Huggingface Models hub as follows:

In [17]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [18]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

The retriever expects tables to be in a particular format. Let's write a function to convert the tables to this format.

In [19]:
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed


Notice that we are only using tables here. However, if you want the retriever to take the metadata into account while retrieving the tables, you can join any metadata strings, such as title, section_title, etc., separated by new line characters at the beginning of the processed table.

Let's take a look at the formatted tables.

In [20]:
# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)
# display the formatted table
processed_tables[2]

'Unnamed: 0,ID,Provider,AddressLine1,AddressLine2,City,State,PostalCode,Phone1_Type,Phone1_Number,Phone2_Type,Phone2_Number,FaxNumber,WebsiteUrl,USEFUL,FINDABLE,USABLE,TOTAL_SCORE,NOTES,Medicare,EmailAddress,ContactName,ContactBusinessTitle,Count_phones,Count_services_before_categorization,Count_services_after_categorization,Count_programs,Count_counties,Count_payments,HOUSING,COMMUNITY SUPPORT & HEALTH SERVICES,OUTDOOR SPACES AND BUILDINGS,RESPECT AND SOCIAL INCLUSION,TRANSPORT,CIVIC PARTICIPATION & EMPLOYMENT,COMMUNICATION & INFORMATION,SOCIAL PARTICIPATION,Elderly and Disabled,Assisted Living,Independent Living,Bridge to Independence,Traumatic Brain Injury / Spinal Cord Injury,Intellectual Disability / Developmental Disability,Private Pay: Credit,Private Pay: Cash,Other,Private Insurance,Medicaid,Private Pay: Check,Tunica,Quitman,Hinds,Tallahatchie,Sharkey,Bolivar,Noxubee,Tishomingo,Adams,Forrest,Itawamba,DeSoto,Tate,Franklin,Jones,Marshall,Panola,Calhoun,Yalobusha,Yazoo,Lowndes,Lef

The formatted table may not make sense to us, but the embedding model is trained to understand it and generate accurate embeddings.

# Initialize Pinecone Index

We will use the Pinecone vector database as our vector index. The Pinecone index stores vector representations of our tables which we can retrieve using a natural language query (query vector). Pinecone does this by computing the similarity between the query vector and the embedded tables stored in the vector index.

To use Pinecone, we first need to initialize a connection to Pinecone. For this, we need a [free API key](https://app.pinecone.io/), and then we initialize the connection like so:

In [21]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.6 MB/s eta 0:00:00


In [22]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='8f40c688-6aa2-4215-89ac-a269e9c4c317')


Now we create a new index. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings outputs 768-dimension vectors. Pinecone will use cosine similarity to compute the similarity between the query and table embeddings.

In [24]:
# you can choose any name for the index
index_name = "table-qa"

# Check if the table-qa index exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)

# Generate Embeddings and Upsert


Next we need to generate the table embeddings and upload it to the Pinecone index. We can easily do that as follows:

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_tables), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_tables))
    # extract batch
    batch = processed_tables[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/28 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2560}},
 'total_vector_count': 2560}

Now the Pinecone index is ready for querying. Let's test to see if it returns tables relevant to our queries.

In [ ]:
query = "What is the address of the provider AAA Ambulance Service*?"
# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(vector=xq, top_k=1)
result


{'matches': [{'id': '1408', 'score': 0.482781589, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

The Pinecone index has returned the ```id``` of a table that would contain the answer to our query with 82.2% confidence. Let's see if this table actually contains the answer. We can use the returned ```id``` as an index to get the relevant pandas dataframe from the ```tables``` list.

In [ ]:
id = int(result["matches"][0]["id"])
tables[id].head()

,Unnamed: 0,ID,Provider,AddressLine1,AddressLine2,City,State,PostalCode,Phone1_Type,Phone1_Number,...,Jefferson Davis,Lee,Covington,Harrison,Coahoma,Pearl River,Chickasaw,Monroe,Wilkinson,Pike
1408,1408,1409,Riemann Family Funeral Home - Kiln/DeLisle,9113 Kiln Delisle Road,nan,Pass Christian,MS,39571,Main,2285860510,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


The table returned by the Pinecone index indeed has the answer to our query. Now we need a model that can read this table and extract the precise answer.

# Initialize Table Reader

As the reader, we will use a TAPAS model fine-tuned for the Table QA task. TAPAS is a BERT-like Transformer model pretrained in a self-supervised manner on a large corpus of English language data from Wikipedia. We load the model and tokenizer from the Huggingface model hub into a question-answering pipeline.

In [ ]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
# load the tokenizer and the model from huggingface model hub
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
# load the model and tokenizer into a question-answering pipeline
pipe = pipeline("table-question-answering",  model=model, tokenizer=tokenizer, device=device)

Let's run the table returned by the Pinecone index and the query we used before into the question-answering pipeline to extract the answer.

In [ ]:
pipe(table=tables[1], query=query)


IndexError: iloc cannot enlarge its target object

The model has precisely answered our query. Let's run some more queries.

# Querying

First, we will define two function to handle our queries and extract answers from tables.

In [ ]:
def query_pinecone(query):
    # generate embedding for the query
    xq = retriever.encode([query]).tolist()
    # query pinecone index to find the table containing answer to the query
    result = index.query(vector=xq, top_k=1)
    # return the relevant table from the tables list
    return tables[int(result["matches"][0]["id"])]

In [ ]:
def get_answer_from_table(table, query):
    # run the table and query through the question-answering pipeline
    answers = pipe(table=table, query=query)
    return answers

In [ ]:
query = "What is the address of the provider AAA Ambulance Service*?"
table = query_pinecone(query)
table

,Unnamed: 0,ID,Provider,AddressLine1,AddressLine2,City,State,PostalCode,Phone1_Type,Phone1_Number,...,Jefferson Davis,Lee,Covington,Harrison,Coahoma,Pearl River,Chickasaw,Monroe,Wilkinson,Pike
1408,1408,1409,Riemann Family Funeral Home - Kiln/DeLisle,9113 Kiln Delisle Road,nan,Pass Christian,MS,39571,Main,2285860510,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
get_answer_from_table(table, query)

IndexError: iloc cannot enlarge its target object

In [ ]:
query = "which scientist is known for improving the steam engine?"
table = query_pinecone(query)
table.head()

,Year,Name,Location,Rationale
0,1839,Robert Hare,"Philadelphia , Pennsylvania",Inventor of the oxy-hydrogen blowpipe
1,1862,John Ericsson,"New York , New York",His work improved the field of heat management...
2,1865,Daniel Treadwell,"Cambridge , Massachusetts",Heat management . He was awarded especially fo...
3,1866,Alvan Clark,"Cambridge , Massachusetts",Improved refracting telescopes
4,1869,George Henry Corliss,"Providence , Rhode Island",For improving the steam engine


In [ ]:
get_answer_from_table(table, query)

{'answer': 'George Henry Corliss',
 'coordinates': [(4, 1)],
 'cells': ['George Henry Corliss'],
 'aggregator': 'NONE'}

In [ ]:
query = "What is the Maldivian island name for Oblu Select at Sangeli	resort?"
table = query_pinecone(query)
table.head()

,Name,Resort Name,Geographic Atoll
0,Asdhoo,Asdu Sun Island Resort,North Male Atoll
1,Akirifushi,Oblu Select at Sangeli,North Male Atoll
2,Baros,Baros Island Resort,North Male Atoll
3,Biyaadhoo,Biyadhoo Island Resort,South Male Atoll
4,Bodubandos,Bandos Maldives Resort,North Male Atoll


In [ ]:
get_answer_from_table(table, query)

{'answer': 'Akirifushi',
 'coordinates': [(1, 0)],
 'cells': ['Akirifushi'],
 'aggregator': 'NONE'}

As we can see, our Table QA system can retrieve the correct table from the Pinecone index and extract precise answers from the table. The TAPAS model we use supports more advanced queries. It has an aggregation head which indicates whether we need to count, sum, or average cells to answer the questions. Let's run some advanced queries that require aggregation to answer.

In [ ]:
query = "what was the total GDP of China and Indonesia in 2020?"
table = query_pinecone(query)
table.head()

,Rank,Country,"GDP ( PPP , Peak Year ) millions of USD",Peak Year
0,1,China,"27,804,953",2020
1,2,India,"11,321,280",2020
2,3,Russia,"4,389,960",2019
3,4,Indonesia,"3,778,134",2020
4,5,Brazil,"3,596,841",2020


In [ ]:
get_answer_from_table(table, query)

{'answer': 'SUM > 27,804,953, 3,778,134',
 'coordinates': [(0, 2), (3, 2)],
 'cells': ['27,804,953', '3,778,134'],
 'aggregator': 'SUM'}

Here the QA system suggests the correct cells to add in order to get the total GDP of China and Indonesia in 2020.

In [ ]:
query = "what is the average carbon emission of power stations in australia, canada and germany?"
table = query_pinecone(query)
table.head()

,CO 2 intensity ( kg/kWh ),Power station,Country
0,1.58,"Hazelwood Power Station , Victoria closed 31 M...",Australia
1,1.56,"Edwardsport IGCC , Edwardsport , Indiana , clo...",United States
2,1.27,"Frimmersdorf power plant , Grevenbroich",Germany
3,1.25,"HR Milner Generating Station , Grande Cache , ...",Canada
4,1.18,"C. TG . Portes Gil , Río Bravo",Mexico


In [ ]:
get_answer_from_table(table, query)

{'answer': 'AVERAGE > 1.58, 1.27, 1.25',
 'coordinates': [(0, 0), (2, 0), (3, 0)],
 'cells': ['1.58', '1.27', '1.25'],
 'aggregator': 'AVERAGE'}

As we can see, the QA system correctly identified which cells to average to answer our question.